In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import datetime
import numpy as np
import pandas as pd

In [9]:
# Parameters of the call
def params(argument):
    if argument == 'team_1':
        return 'Natus Vincere'
    elif argument == 'team_2':
        return 'fnatic'
    elif argument == '_map':
        return 'Mirage'
    elif argument == 'rank_1':
        return 867
    elif argument == 'rank_2':
        return 583
    raise NameError

In [10]:
# Importing the dataset
print('Importing data...')
dataset = pd.read_csv('../data-collecter-js/csv/results.csv')
dataset['date'] = pd.to_datetime(dataset['date'])
_2019 = datetime.date(2019, 1, 1)
_2019 = pd.to_datetime(_2019)
dataset = dataset.loc[dataset['date'] >= _2019]
dataset_p1 = dataset.loc[dataset['team_1'] == params('team_1')]
dataset_p2 = dataset.loc[dataset['team_2'] == params('team_1')]
dataset_p3 = dataset.loc[(dataset['team_1'] == params('team_2')) & (dataset['team_2'] != params('team_1'))]
dataset_p4 = dataset.loc[(dataset['team_2'] == params('team_2')) & (dataset['team_1'] != params('team_1'))]
dataset = pd.concat([dataset_p1, dataset_p2, dataset_p3, dataset_p4])
X = dataset.iloc[:, [1, 2, 3, 14, 15]].values
y = dataset.iloc[:, 6].values
print('Success!')

Importing data...
Success!


In [11]:
# Start function
def choice():
    print('Starting to learn...')
    user_choice = '1'
    if user_choice == '1':
        learn()
    elif user_choice == '2':
        print('Programm finished.')
    else:
        print('\nYou probably pressed the wrong button, try again!')
        choice()

In [12]:
# Learning accuracy function, %
def accuracy():
    global X
    global y
    print('Evaluating accuracy...')
    # Encoding categorical data
    onehotencoder = OneHotEncoder()
    X_new = onehotencoder.fit_transform(X[:, 0:2]).toarray()
    X_new = X_new[:, 1:]
    X = X[:, [3, 4]]
    X = np.concatenate((X, X_new), axis=1)

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

    # Fitting XGBoost
    classifier = XGBClassifier(n_estimators=500, learning_rate=0.00001,
                               max_depth=7, subsample=0.8, colsample_bytree=0.8, gamma=5)
    classifier.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    cm_sum = cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1]
    avg_acc = (cm[0][0] + cm[1][1]) / cm_sum * 100
    # Returning average accuracy of the algorithm
    return avg_acc

In [13]:
# Main predict function
def learn():
    global X
    global y
    team_1 = params('team_1')
    team_2 = params('team_2')
    rank_1 = params('rank_1')
    rank_2 = params('rank_2')
    _map = params('_map')
    X_test = np.array([[team_1, team_2, _map, rank_1, rank_2]])
    # Encoding categorical data
    print('Encoding data...')
    onehotencoder = OneHotEncoder()
    X_new_transformed = onehotencoder.fit_transform(X[:, 0:2]).toarray()
    X_new_transformed = X_new_transformed[:, 1:]
    X_transformed = X[:, [3, 4]]
    X_transformed = np.concatenate((X_transformed, X_new_transformed), axis=1)
    X_test_new_transformed = onehotencoder.transform(X_test[:, 0:2]).toarray()
    X_test_new_transformed = X_test_new_transformed[:, 1:]
    X_test_transformed = X_test[:, [3, 4]]
    X_test_transformed = np.concatenate((X_test_transformed, X_test_new_transformed), axis=1)
    print('Success!')
    print('Splitting the dataset...')
    # Splitting the dataset into the Training set and Test set
    X_train, X_test_old, y_train, y_test_old = train_test_split(X_transformed, y, test_size=1)
    print('Success!')
    print('Fitting XGBoost...')
    # Fitting XGBoost
    classifier = XGBClassifier(n_estimators=500, learning_rate=0.00001,
                               max_depth=7, subsample=0.8, colsample_bytree=0.8, gamma=5)
    classifier.fit(X_train, y_train)
    print('Success!')
    print('Predicting')
    # Predicting
    y_pred = classifier.predict(X_test_transformed)
    print('Success!')
    accuracy_final = accuracy()
    # Predicting the Test set results
    print(f'Accuracy of the algorithm: {accuracy_final}%')
    if y_pred == 2:
        print(f'Algorithm predicts that {team_1} will win {team_2}.')
    elif y_pred == 1:
        print(f'Algorithm predicts that {team_2} will win {team_1}.')
    else:
        print('An unexpected error has occured.')
    print('Programm finished.')

In [14]:

choice()

Starting to learn...
Encoding data...
Success!
Splitting the dataset...
Success!
Fitting XGBoost...
[21:18:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/brian/.conda/envs/workspace/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Success!
Predicting


XGBoostError: [21:18:34] ../src/c_api/../data/array_interface.h:357: Unicode-3 is not supported.
Stack trace:
  [bt] (0) /home/brian/.conda/envs/workspace/lib/libxgboost.so(+0x9c2d5) [0x7fd274e9c2d5]
  [bt] (1) /home/brian/.conda/envs/workspace/lib/libxgboost.so(+0xac3c6) [0x7fd274eac3c6]
  [bt] (2) /home/brian/.conda/envs/workspace/lib/libxgboost.so(XGBoosterPredictFromDense+0x11d) [0x7fd274eaef0d]
  [bt] (3) /home/brian/.conda/envs/workspace/lib/python3.10/lib-dynload/../../libffi.so.7(+0x69dd) [0x7fd2b58e29dd]
  [bt] (4) /home/brian/.conda/envs/workspace/lib/python3.10/lib-dynload/../../libffi.so.7(+0x6067) [0x7fd2b58e2067]
  [bt] (5) /home/brian/.conda/envs/workspace/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x13a25) [0x7fd2b54f2a25]
  [bt] (6) /home/brian/.conda/envs/workspace/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x1049f) [0x7fd2b54ef49f]
  [bt] (7) /home/brian/.conda/envs/workspace/bin/python(_PyObject_MakeTpCall+0x15e) [0x